# Fix LAMMPS versions

Support for most interatomic potentials in LAMMPS has stayed consistent over the years, but occasionally updates in LAMMPS have resulted in certain potential implementations no longer working.  This Notebook provides a means of changing the LAMMPS version to use on prepared calculations so that those out-of-date implementations can be tested.

__NOTE:__ In almost all cases, users should *always* select the most recent active implementation hosted in the repository.  This Notebook is primarily used by the NIST Interatomic Potentials Repository to verify if the different versions of the potentials behave identically in the versions of LAMMPS that support them.

The newest versions of the potentials hosted should be compatible with the newest versions of LAMMPS.  If you find this not to be the case, please let us know by sending an email to [potentials@nist.gov](potentials@nist.gov).

In [1]:
from pathlib import Path

import iprPy

## Define settings

ALL PATHS ARE MACHINE DEPENDENT! SET USING WHAT YOU HAVE ON YOUR COMPUTER!

In [2]:
# Pick the run directory to modify
run_directory = iprPy.load_run_directory('master_1')

# Define the current lammps executable path string to replace
oldlammps = 'lmp_mpi'

# Initialize replacementdict
replacementdict = {}

### SNAP potentials

SNAP potentials have 4 variations:
- 1 needs LAMMPS between 8 Oct 2014 and 30 May 2017
- 2a needs LAMMPS between 30 May 2017 and 3 Dec 2018
- 2 needs LAMMPS between 3 Dec 2018 and 12 June 2019
- 3 needs LAMMPS after 12 June 2019

1 needs LAMMPS between 8 Oct 2014 and 30 May 2017

In [3]:
#lmp_snap_1 = 'C:/Program Files/LAMMPS/2017-01-27/bin/lmp_mpi'
lmp_snap_1 = 'E:/LAMMPS/2017-01-27/bin/lmp_mpi'

snap_1_potentials = ['2015--Thompson-A-P--Ta--LAMMPS--ipr1']

for pot_id in snap_1_potentials:
    replacementdict[pot_id] = lmp_snap_1

In [4]:
#lmp_snap_2 = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'
lmp_snap_2 = 'E:/LAMMPS/2019-06-05/bin/lmp_mpi'

snap_2_potentials = ['2015--Thompson-A-P--Ta--LAMMPS--ipr2']

for pot_id in snap_2_potentials:
    replacementdict[pot_id] = lmp_snap_2

### Fail robust file checks

The parameter file interpreters were updated in the Oct 30 2019 LAMMPS patch to perform more rigorous format checks.  A number of the older implementations hosted on the repository will not work for LAMMPS versions after that date.

In [5]:
#lmp_old = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'
lmp_old = 'E:/LAMMPS/2019-06-05/bin/lmp_mpi'
old_potentials = [
    '1987--Ackland-G-J--Mo--LAMMPS--ipr1',
    '1987--Ackland-G-J--Mo--LAMMPS--ipr2',
    '2011--Bonny-G--Fe-Cr--LAMMPS--ipr1',
    '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1',
    '2013--Smirnova-D-E--U-Mo-Xe--LAMMPS--ipr1',
]
 
for pot_id in old_potentials:
    replacementdict[pot_id] = lmp_old

In [6]:
def changelammpsversions(run_directory, replacementdict, oldlammps):
    """
    Iterates over all prepared calculations in a run_directory and updates
    the LAMMPS version to use.
    
    Parameters
    ----------
    run_directory : Path
        The run directory containing prepared calculations to update.
    replacementdict : dict
        keys are the LAMMPS potential ids and values are the Paths to the
        LAMMPS executables to use.
    oldlamms : str
        The current lammps executable to replace with the correct path.    
    """
    for inscript in run_directory.glob('*/calc_*.in'):
        with open(inscript) as f:
            content = f.read()
        
        for pot_id, lammps in replacementdict.items():
            if pot_id in content:
                content = content.replace(oldlammps, lammps)
                break
                
        with open(inscript, 'w') as f:
            f.write(content)

In [7]:
changelammpsversions(run_directory, replacementdict, oldlammps)